# Binary Text Classification

### Source
Algorithm was copy & pasted from this [Medium article](https://bmanikan.medium.com/text-classification-a-parameter-free-method-with-compressors-b96cd151adbb)

In [4]:
import gzip
import numpy as np
import pandas as pd

DATA_DIR = 'data'
test = pd.read_csv(f'{DATA_DIR}/test.csv')
train = pd.read_csv(f'{DATA_DIR}/train.csv')

test_set = test[['OriginalTweet','Sentiment']]
training_set = train[['OriginalTweet','Sentiment']]

In [5]:
test_set = [(row['OriginalTweet'],row['Sentiment']) for idx, row in test_set.iterrows()]
training_set = [(row['OriginalTweet'],row['Sentiment']) for idx, row in training_set.iterrows()]

In [9]:
# Copy paste from: https://bmanikan.medium.com/text-classification-a-parameter-free-method-with-compressors-b96cd151adbb
import gzip
import numpy as np

# Define the function to calculate NCD
def calculate_ncd(x1, x2):
    Cx1 = len(gzip.compress(x1.encode()))
    Cx2 = len(gzip.compress(x2.encode()))
    x1x2 = " ".join([x1, x2])
    Cx1x2 = len(gzip.compress(x1x2.encode()))
    ncd = (Cx1x2 - min(Cx1, Cx2)) / max(Cx1, Cx2)
    return ncd

# Define the function for k-NN classification
def knn_classify(test_instance, training_set, k):
    distance_from_test_instance = []
    for (x2, _) in training_set:
        ncd = calculate_ncd(test_instance, x2)
        distance_from_test_instance.append(ncd)
    sorted_idx = np.argsort(np.array(distance_from_test_instance))
    top_k_class = [training_set[i][1] for i in sorted_idx[:k]]
    predict_class = max(set(top_k_class), key=top_k_class.count)
    return predict_class

# # Example texts
# test_set = [
#     ("soccer is wonderful game", _),
#     ("I love pizza man", _)
# ]

# training_set = [
#     ("I love pizza", "Class A"),
#     ("Ice cream is delicious", "Class A"),
#     ("Basketball requires good coordination", "Class B"),
#     ("Soccer is a wonderful game", "Class B"),
#     ("Tennis need efforts", "Class B")
# ]

k = 5  # Number of neighbors to consider

# Apply the k-NN classification to the test set
for (test_instance, _) in test_set[:3]:
    predicted_class = knn_classify(test_instance, training_set, k)
    print(f"'{_}&&{predicted_class}&&{test_instance}''\n{'='*100}")

'negative&&positive&&TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1''
'positive&&positive&&When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY''
'negative&&positive&&#Panic buying hits #NewYork City as anxious shoppers stock up on food&amp;medical supplies after #healthcare worker in her 30s becomes #BigApple 1st confirmed #coronavirus patient OR a #Bloomberg staged event?

https://t.co/IASiReGPC4

#QAnon #QAnon2018 #QAnon2020 
#Election2020 #CDC https://t.co/29isZOewxu''


In [ ]:
# Now do the same thing but with sklearn. Hopefully it's faster/more accurate?
# Use GridsearchCV https://towardsdatascience.com/cross-validation-and-grid-search-efa64b127c1b